In [ ]:
import twint
file = open("../security_tags.txt","r")
arr = []
line1 = file.readlines()
for x in line1:
    arr.append(x)
file.close()    

In [92]:
for i in range(len(arr)):
    arr[i] = arr[i][1:]

In [131]:
file = open("./test1.json","w")
file.close()

In [130]:
# # clear file
# #Twint Search
# for i in range(len(arr)):
#     d = twint.Config()
#     d.Search = arr[i]
#     d.Limit = 50
#     # d.Store_csv = True
#     d.Store_object = True
#     d.Store_json = True
#     d.Output = "./test1.json"
#     # d.Database = "tweets.db"


#     x = twint.run.Search(d)

In [ ]:
# clear file
# Twint Search
d = twint.Config()
d.Search = "virustotal"
# d.Search = "endpoint AND seminar"
d.Limit = 1000
# d.Store_csv = True
d.Store_object = True
d.Store_json = True
d.Output = "./test1.json"
# d.Database = "tweets.db"

x = twint.run.Search(d)

In [114]:
# Search terms
# 0-day AND exploit
# “0-day”, “CVE-“, “CVE-2018-*”, “CVE-2020-*”
# usernames:
# kibbsy
# it_securitynews
# msftsecurity

In [115]:
# x = twint.run.Search(d)
# twint -u noneprivacy --csv --output "/Users/psingh4/harsh/test4.json" --lang en --translate --translate-dest it --limit 100

In [116]:
# d.__dict__.keys()

In [117]:
# twint.run.Search(d)
# twint.output

In [118]:
################################################################################################
# Loads file written from Twint
################################################################################################

f= open("./test1.json","r",errors='ignore')
line1 = f.readlines()
arr = []
for x in line1:
    arr.append(x)
f.close()    

In [119]:
################################################################################################
# Fixes Error from Twint where braces and commas were missing in the json
################################################################################################
arr.insert(0,"[")
arr.append(']')
for i in range(1,len(arr)-2):
    arr[i] = arr[i][:-1] + "," + arr[i][-1:]

In [120]:
################################################################################################
# Writes the corrected files to json
################################################################################################
file = open("../test.json","w")
for i in range(len(arr)):
    file.write(arr[i])
file.close()

In [121]:
################################################################################################
# Loads corrected JSON file
################################################################################################

f= open("../test.json","r",errors='ignore')
line1 = f.readlines()
data = []
for x in line1:
    data.append(x)
f.close()

In [122]:
################################################################################################
# Removes Non-ASCII characters
################################################################################################
for i in range(len(data)):
    data[i] = (''.join([i if ord(i) < 128 else ' ' for i in str(data[i])]))

In [123]:
file = open("./data.json","w")
for i in range(len(data)):
    file.write(data[i])
file.close()

In [71]:
import json
with open("./data.json", "r",errors='ignore') as read_file:
    data = json.load(read_file)

In [98]:
data[130]['tweet']
# is_english(str(data[130]['tweet']))

'id::1220037979818856448:RT @cglyer: BREAKING - To help organizations identify compromised systems with CVE-2019-19781, @FireEye  &amp; @Citr'

In [36]:
translator.detect("Hello").lang == 'en'


ConnectionError: ('Connection aborted.', OSError("(54, 'ECONNRESET')"))

In [37]:
################################################################################################
# Google Trans model for checking if a tweet is english or not, unpoisons the training data
################################################################################################

# ! pip install googletrans
from googletrans import Translator
translator = Translator(service_urls=['translate.google.com'])

def is_english(sentence):
    if translator.detect(sentence).lang == 'en':
        return True
    else:
        return False
    
with open("../classified_tweets.json", "r",errors='ignore') as read_file:
    data = json.load(read_file)

i=0
skip = [130]
while i <= len(data):
#     print(i)
    if (i in skip): continue
    if is_english(data[i]['tweet']) == False:
        print(data[i]['tweet'])
        data.pop(i)
    else: i+= 1

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [126]:
################################################################################################
# MySql connection established
################################################################################################

import mysql.connector
mydb = mysql.connector.connect(user='admin', password='Private2712!',
                              host='database-1.cok63qqiofsd.us-east-1.rds.amazonaws.com',
                              database='data')
mycursor = mydb.cursor()

In [128]:
###############################################################################################################
# Insert records from Twint into Relation DB, see relation schema code on GitHub for table structure
# Requires the JSON file with metadata
###############################################################################################################

sql = """INSERT INTO classified_tweets (relevant, created_at, conversation_id, id, date, time, timezone, user_id, username, name, place, tweet, mentions, urls, replies_count, 
        retweets_count, likes_count, hashtags, cashtags, link, retweet, video, near, geo, source, user_rt_id, user_rt, retweet_id,
        retweet_date, translate, trans_src, trans_dest, photos, reply_to) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

val = []
for i in range(1, len(data)):
    val.append((1, data[i]['created_at'], data[i]['conversation_id'], str(data[i]['id']), data[i]['date'], data[i]['time'], data[i]['timezone'], data[i]['user_id'], data[i]['username'],
      data[i]['name'], data[i]['place'], data[i]['tweet'], str(data[i]['mentions']), str(data[i]['urls']), data[i]['replies_count'],
      data[i]['retweets_count'], data[i]['likes_count'], str(data[i]['hashtags']), str(data[i]['cashtags']), data[i]['link'], data[i]['retweet'],
      data[i]['video'], data[i]['near'], data[i]['geo'], data[i]['source'], data[i]['user_rt_id'], data[i]['user_rt'],
      data[i]['retweet_id'], data[i]['retweet_date'], data[i]['translate'], data[i]['trans_src'], data[i]['trans_dest'],str(data[i]['photos']), 
       str(data[i]['reply_to'])))
    
mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "record inserted.")

mydb.close()

1004 record inserted.


In [ ]:
###############################################################################################################
# Cell to see how the data looks at in val (Outdated, more attributes being captured now)
###############################################################################################################

val = []
for i in range(len(data)):
    val.append((data[i]['created_at'], data[i]['conversation_id'], data[i]['date'], data[i]['time'], data[i]['timezone'], data[i]['user_id'], data[i]['username'],
      data[i]['name'], data[i]['place'], data[i]['tweet'], str(data[i]['mentions']), str(data[i]['urls']), data[i]['replies_count'],
      data[i]['retweets_count'], data[i]['likes_count'], str(data[i]['hashtags']), str(data[i]['cashtags']), data[i]['link'], data[i]['retweet'],
      data[i]['video'], data[i]['near'], data[i]['geo'], data[i]['source'], data[i]['user_rt_id'], data[i]['user_rt'],
      data[i]['retweet_id'], data[i]['retweet_date'], data[i]['translate'], data[i]['trans_src'], data[i]['trans_dest'],0,str(data[i]['photos']), 
       str(data[i]['reply_to'])))

In [ ]:
# mydb.commit()
# !pip install tensorflow
# print(mycursor.rowcount, "record inserted.")

In [ ]:
# !pip install bert-serving-server
# !pip install bert-serving-client

In [5]:
import nltk
sentence = "NSA exploit EternalRomance with CVE-2019-897 'formed basis for alleged Chinese tool' - iTWire Researchers Mark Lechtik and Nadav Grossman said in first offered NSA exploits that the hacking group, known variously as APT3 and Gothic Panda, had based its exploitation tool, https://ift.tt/2HRG9js"
# sentence = "NowBrowsing: ESLint dependencies are vulnerable (ReDoS and Prototype Pollution) · CVE-2020-7598 · GitHub Advisory Database:  https://github.com/advisories/GHSA-7fhm-mqm4-2wp7"
tokens = nltk.word_tokenize(sentence)
tagged = nltk.pos_tag(tokens)
tagged

In [7]:
# !python -m spacy download en
# !python -m spacy download en_core_web_sm

import spacy
nlp = spacy.load('en_core_web_sm')

In [9]:
# sentence = "NSA exploit EternalRomance with CVE-2019-897 'formed basis for alleged Chinese tool' - iTWire Researchers Mark Lechtik and Nadav Grossman said in first offered NSA exploits that the hacking group, known variously as APT3 and Gothic Panda, had based its exploitation tool, https://ift.tt/2HRG9js"
sentence = "NowBrowsing: ESLint dependencies are vulnerable (ReDoS and Prototype Pollution) · CVE-2020-7598 · GitHub Advisory Database:  https://github.com/advisories/GHSA-7fhm-mqm4-2wp7"
doc = nlp(sentence)
print("The sentence is: ", sentence)
print([ent for ent in doc.ents])
# for ent in doc.ents:
# doc

The sentence is:  NowBrowsing: ESLint dependencies are vulnerable (ReDoS and Prototype Pollution) · CVE-2020-7598 · GitHub Advisory Database:  https://github.com/advisories/GHSA-7fhm-mqm4-2wp7
[NowBrowsing, CVE-2020-7598]


In [ ]:
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])

In [ ]:
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

In [ ]:
# !pip install -U bert-serving-server bert-serving-client

In [ ]:
# Run on termninal if Bert Server and Client installed
# !bert-serving-start -model_dir /tmp/english_L-12_H-768_A-12/ -num_worker=4 

In [ ]:
# from bert_serving.client import BertClient
# bc = BertClient()
# bc.encode(['First do it', 'then do it right', 'then do it better'])

In [ ]:
# first neural network with keras tutorial
import numpy as np
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
# load the dataset
data = np.load('./tweet_encodings.npy')
label = np.load('./maybeincludedlabels.npy')
# define the keras model
model = Sequential()
model.add(Dense(400, input_dim=768, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(2, activation='softmax'))
# compile the keras model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
# evaluate the keras model

In [ ]:
model.fit(data, label, epochs=10, batch_size=1, validation_split=0.25)

In [ ]:
import sklearn
from sklearn.linear_model import LogisticRegression
len(data), len(label)

In [ ]:
clf = LogisticRegression(random_state=1).fit(data[:-900], label[:-900])

In [ ]:
clf.score(data[-900:], label[-900:])

In [ ]:
y_pred = clf.predict(data[-800:])

In [ ]:
y_true = label[-800:]

In [ ]:
from sklearn import metrics

accuracy_score = metrics.accuracy_score(y_true,y_pred)
print("Accuracy score: {}".format(accuracy_score))
fpr, tpr, thresholds = metrics.roc_curve(y_true,y_pred,pos_label = 1)
auc_score = metrics.auc(fpr,tpr)
print("AUC: {}".format(auc_score))


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import metrics
import numpy as np
import sys
import pdb
import matplotlib.pyplot as plt


def print_metrics(y_true,y_pred):
    accuracy_score = metrics.accuracy_score(y_true,y_pred)
    print("Accuracy score: {}".format(accuracy_score))
    fpr, tpr, thresholds = metrics.roc_curve(y_true,y_pred,pos_label = 1)
    auc_score = metrics.auc(fpr,tpr)
    print("AUC: {}".format(auc_score))
    return accuracy_score,auc_score

def logistic_regression(x_train,x_test,y_train,y_test):
    logisticRegr = LogisticRegression()
    logisticRegr.fit(x_train,y_train)
    predictions = logisticRegr.predict(x_test)
    print_metrics(y_test,predictions)


def support_vector(x_train,x_test,y_train,y_test):
    model = svm.SVC()
    model.fit(x_train,y_train)
    predictions = model.predict(x_test)
    print_metrics(y_test,predictions)

def torch_label_creator(label):
    if label.item() == 1:
        return torch.tensor([1,0])
    else:
        return torch.tensor([0,1])

    #plot test accuracies and auc over epochs    
    plt.plot(test_accuracies)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.title('Test Accuracy vs Epoch')
    plt.show()
    plt.plot(test_auc)
    plt.ylabel('AUC')
    plt.xlabel('Epoch')
    plt.title('Test AUC vs Epoch')
    plt.show()

In [ ]:
encodings = np.load('tweet_encodings.npy')
labels = np.load('maybeincludedlabels.npy')
x_train,x_test,y_train,y_test = train_test_split(encodings,labels, test_size=0.25, random_state=0)

In [ ]:
logistic_regression(x_train,x_test,y_train,y_test)

In [ ]:
clf.score(x_test, y_test)

In [ ]:
from neo4j import GraphDatabase
import json
#Indicate to driver where neo4j is running
driver = GraphDatabase.driver("bolt://localhost",auth=("test_user","password"), encrypted=False)
#Parse data
counter = 0
with open("./data.json",'r') as file:
    #Need to make a session where you will run all your cypher queries
    with driver.session() as session:
        tx = session.begin_transaction()
        noun_count = 0
        count = 0
        for line in file.readlines(): #can limit lines with [:100] after ()
            if (line == "[\n" or line == "]"):
                continue
            item = json.loads(line[:-2])
            sen_noun = noun[noun_count]
            # print(item)
            tx.run('''CREATE (a:Tweet{id:$value.id,date:$value.date,train_id:$value.train_id,nouns:$sen_noun})''',
                   parameters={'tweet': item}, value=item, sen_noun=sen_noun) # add parameters here
            #Batch processing to run 1000 tweets as a time as these commits are quite time intensive
            count += 1
            noun_count += 1
#             tx.commit()
#             break   # UNCOMMENT IF PUSHING MULTIPLE ROWS/NODES OF DATA 
            if count > 1000:
                tx.commit()
                tx = session.begin_transaction()
                count = 0
        tx.commit()
# import json
# with open("./data.json", "r") as read_file:
#     for line in read_file.readlines():
#         print(line)
#         if (line == "[\n" or line == "]"):
#             continue
#         item = json.loads(line[:-2])
#         print(item)

In [ ]:
########################################################################################################
# Cypher code for graph db (Some example queries)
########################################################################################################

# Match (a:Tweet) where 'Malware' in a.nouns return a

# Match (a:Tweet), (b:Tweet) where 'Malware' in a.nouns and 'Malware' in b.nouns create (a)-[r:Malware]->(b) return r

# '''WITH {tweet} AS Tweet
#     Merge (a:Tweet{id:$value.id,date:value.date,train_id:value.train_id})
#     Merge (d:Date{date:value.date})
#     Merge (a)-[:SAME_DATE]->(d)'''

# '''CREATE (a:Tweet{id:$value.id,date:$value.date,train_id:$value.train_id,nouns=sen_noun})'''

# '''start n=node(*) return n'''

# MATCH (n:Tweet)-[m:Malware]-(b:Tweet)
#     WHERE n.id = 1223562341540859904
#     REturn b
#     order by b.date

In [17]:
########################################################################################################
# Connection to graph db (Neo4j) established
########################################################################################################

from neo4j import GraphDatabase
import json
#Indicate to driver where neo4j is running
driver = GraphDatabase.driver("bolt://localhost:7687",auth=("test_user","password"), encrypted=False)

In [5]:
import json

data = {}
with open("./datasets/complete_data.json") as f:
    data = json.load(f)

In [6]:
# Note, apply some more feature engineering such as removing stop words from a sentence
# techniques laid out here: https://www.analyticsvidhya.com/blog/2019/08/how-to-remove-stopwords-text-normalization-nltk-spacy-gensim-python/

import spacy
nlp = spacy.load('en_core_web_sm')
sentence = "CVE-2019-897 NSA exploit EternalRomance 'formed basis for alleged Chinese tool' - iTWire Researchers Mark Lechtik and Nadav Grossman said in first offered NSA exploits that the hacking group, known variously as APT3 and Gothic Panda, had based its exploitation tool, https://ift.tt/2HRG9js"
doc = nlp(sentence)
print("The sentence is: ", sentence)
# print([ent for ent in doc.ents])
# for ent in doc.ents:
# doc
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])

The sentence is:  CVE-2019-897 NSA exploit EternalRomance 'formed basis for alleged Chinese tool' - iTWire Researchers Mark Lechtik and Nadav Grossman said in first offered NSA exploits that the hacking group, known variously as APT3 and Gothic Panda, had based its exploitation tool, https://ift.tt/2HRG9js
Noun phrases: ['CVE-2019-897 NSA', "EternalRomance 'formed basis", "alleged Chinese tool' - iTWire Researchers Mark Lechtik", 'Nadav Grossman', 'the hacking group', 'APT3', 'Gothic Panda', 'its exploitation tool', 'https://ift.tt/2HRG9js']


In [7]:
########################################################################################################
# Uses Spacy for finding nouns, named entity recognition
# Old results from spacy, without removing stop words
########################################################################################################

import spacy
from spacy.lang.en import English

# Option = 1 => Get all Nouns, Oprion = 2 => Get position entities
def get_all_nouns( data, option ):
    nlp = spacy.load('en_core_web_sm')
    sentence = data['text'].replace("-", " ").replace("#", " ")
    doc = nlp(sentence)
    if option == 1:
        tagged = [chunk.text for chunk in doc.noun_chunks]
        return tagged
    else:
        pos_ent = []
        for ent in doc.ents:
            pos_ent.append(ent.text)
#             print(ent.text, ent.label_)
        return pos_ent

########################################################################################################
# Uses Spacy for finding nouns, named entity recognition
########################################################################################################
# import spacy

# nlp = spacy.load('en_core_web_sm')

# noun = []
# noun_from_tweet = []
# for i in range(len(data)):
#     if data[i]['relevant'] == 0:
#         continue
#     sen_noun = []
#     sentence = data[i]['text']
#     doc = nlp(sentence)
#     tagged = [chunk.text for chunk in doc.noun_chunks]
#     noun_from_tweet.append(tagged)
#     for nouns in tagged:
#         noun.append(nouns)
# #     noun.append(sen_noun)

In [8]:
########################################################################################################
# Uses Spacy for finding nouns, named entity recognition
# NEW results from spacy, after removing stop words
########################################################################################################

def get_nouns_wo_stop_words (data, option):
    nlp = English()

    my_doc = nlp(data['text'].replace("-", " ").replace("#", " "))
    # Create list of word tokens
    token_list = []
    for token in my_doc:
        token_list.append(token.text)
    from spacy.lang.en.stop_words import STOP_WORDS

    # Create list of word tokens after removing stopwords
    filtered_sentence =[] 

    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word)    # Contains the sentence without stop words
    # print(token_list)
    # print(filtered_sentence) 

    new_text = filtered_sentence[0] + " "
    for i in range(1,len(filtered_sentence)):
        new_text += filtered_sentence[i] + " "

    nlp = spacy.load('en_core_web_sm')

    #  "nlp" Object is used to create documents with linguistic annotations.
    noun = []
    sen_noun = []
    sentence = new_text
    doc = nlp(sentence)
    if option == 1:
        tagged = [chunk.text for chunk in doc.noun_chunks]
        return tagged
    else:
        pos_ent = []
        for ent in doc.ents:
            pos_ent.append(ent.text)
#             print(ent.text, ent.label_)
        return pos_ent

In [9]:
########################################################################################################
# Uses NLTK for finding nouns
########################################################################################################

# nltk.download()    #Crashes on Mac, check on win machine
# Bottom two fixes the issue by not using nltk
# nltk.download('averaged_perceptron_tagger')
# nltk.download('english')

# This cell for nltk

import nltk

def nltk_nouns(data):
    nlp = spacy.load('en_core_web_sm')
    sen_noun = []
    sentence = data['text']
    tokens = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag(tokens)
    for j in range(len(tagged)):
        if (tagged[j][1] == 'NNP'):
            sen_noun.append(tagged[j][0])
    return sen_noun


In [15]:
##############################################################################################################
# Test between with and without stop words noun results for spacy
##############################################################################################################

# Collecting 100 tweets
counter = 0
data_list = []
for i in range(len(data)):
    if data[i]['relevant'] == 1:
        data_list.append(data[i])
        counter += 1
    if (counter == 100): break
# data[2788]
# get_all_nouns(data[2788])

results = []
for i in range(100):
    results.append((get_all_nouns(data_list[i], 2), get_nouns_wo_stop_words(data_list[i], 2)))

In [30]:
results[30:50]
# Note have all occurences of zero day collapse to 0-day node in graph db

[([], []),
 (['LokiBot', 'AgentTesla'], ['AgentTesla']),
 ([], []),
 (['VB6', '@JAMESWT_MHT @wwp96 @unpacme'],
  ['VB6', '@JAMESWT_MHT @wwp96 @unpacme']),
 (['LokiBot', 'AgentTesla'], ['AgentTesla']),
 (["last week's",
   'Emotet 255',
   '270',
   '235',
   '211',
   'AgentTesla',
   '167',
   '325',
   '125'],
  ['270', '235', '211', '167', '325']),
 (['Agenttesla', 'UK Atomic Energy&lt;dav.langridge@ryanplastics.co.uk&gt'],
  ['Agenttesla', 'UK Atomic Energy&lt;dav.langridge@ryanplastics.co.uk&gt']),
 (['Agenttesla', 'UK Atomic Energy&lt;dav.langridge@ryanplastics.co.uk&gt'],
  ['Agenttesla', 'UK Atomic Energy&lt;dav.langridge@ryanplastics.co.uk&gt']),
 (['Agenttesla',
   'UK Atomic Energy&lt;dav.langridge@ryanplastics.co.uk&gt',
   '24',
   '05:38:37',
   '0800',
   'Exfil',
   'klai.epicbrokers[.]com'],
  ['UK Atomic Energy&lt;dav.langridge@ryanplastics.co.uk&gt',
   '24',
   '05:38:37   0800',
   'Exfil',
   'klai.epicbrokers[.]com']),
 ([], []),
 ([], []),
 (['https://t.co/E9lvv

In [25]:
data_list[7]

{'text': 'RT @DBMCSR: #Microsoft Releases Advisory on Zero-Day #Vulnerability CVE-2020-0674, Workaround Provided\nhttps://t.co/61dGXiTHhO\n#0Day #ZeroD…',
 'relevant': 1}

In [25]:
##############################################################################################################
# Pulling out every hashtag and @mentions from a Tweet
##############################################################################################################

import re

# Data is only a string, not dictionary element
def ret_hash_at (data):
    hashes = re.findall('#\w+', data)   # #\w+ is a regular expression for removing words starting with #
    ats = re.findall('@\w+', data)      # @\w+ is a regular expression for removing words starting with @
    # m = re.search('#\w+', data_list[28]['text'])  # Search used for checking existence, stops after finding first match, will skip any subsequent parts of the string
    # m.group(0)
    return (hashes, ats)

{'text': 'RT @DBMCSR: #Microsoft Releases Advisory on Zero-Day #Vulnerability CVE-2020-0674, Workaround Provided\nhttps://t.co/61dGXiTHhO\n#0Day #ZeroD…',
 'relevant': 1}

In [ ]:
##############################################################################################################
# Print results for comparing w/wo spacy
##############################################################################################################

# results[28]
(get_all_nouns(data_list[28]), get_nouns_wo_stop_words(data_list[28]))

In [46]:
x = set()
for i in range(len(results[28][0])):
    x.add(results[28][0][i])
for i in range(len(results[28][1])):
    x.add(results[28][1][i])
x

{'Adwind Massive Ongoing Phishing Campaign https://t.co/XNVzXgWVjP',
 'Turkish Rat',
 'a Massive Ongoing Phishing Campaign https://t.co/XNVzXgWVjP',
 '“The Turkish Rat” Evolved Adwind'}

In [112]:
print(data_list[26])
print(data_list[28])
nlp = spacy.load('en_core_web_sm')
noun = []
sen_noun = []
sentence = data_list[26]['text'].replace("-", " ").replace("#", " ")
doc = nlp(sentence)
tagged = [chunk.text for chunk in doc.noun_chunks]
# ADD BELOW TEXT TO FUNCTIONS (MAKE FEATURE FOR NOUNS OR NAMED ENTITIES)
for ent in doc.ents:
    print(ent.text, ent.label_)
print(tagged)


{'text': 'Security researchers spotted the latest iteration of an evolving Turkish phishing campaign that’s targeting more than 80 companies with Adwind malware. https://t.co/k02mPIz17R @IBMSecurity #Adwind #malware https://t.co/P45lVKAcpY', 'relevant': 1}
{'text': '“The Turkish Rat” Evolved Adwind in a Massive Ongoing Phishing Campaign https://t.co/XNVzXgWVjP @CheckPointSW #Adwind #malware https://t.co/7XUqabwEYQ', 'relevant': 1}
Turkish NORP
more than 80 CARDINAL
Adwind GPE
Adwind GPE
['Security researchers', 'the latest iteration', 'an evolving Turkish phishing campaign', 'more than 80 companies', 'Adwind malware', 'Adwind  malware https://t.co/P45lVKAcpY']


In [108]:
data[28]

{'train_id': 3354,
 'relevant': 1,
 'created_at': 1579737681000,
 'date': '2020-01-22',
 'time': '16:01:21',
 'timezone': 'Pacific Standard Time',
 'user_id': 2147483647,
 'username': 'vulmonfeeds',
 'name': 'Vulmon Vulnerability Feed',
 'place': '',
 'tweet': 'CVE-2019-19841\n\nemfd in Ruckus Wireless Unleashed through 200.7.10.102.64 allows remote attackers to execute OS commands via a POST request with the attribute xcmd=packet-capture to admin/_cmdstat.jsp via the mac at...\n\n http://vulmon.com/vulnerabilitydetails?qid=CVE-2019-19841     ',
 'mentions': '[]',
 'urls': "['http://vulmon.com/vulnerabilitydetails?qid=",
 'replies_count': 0,
 'retweets_count': 0,
 'likes_count': 0,
 'hashtags': '[]',
 'cashtags': '[]',
 'link': 'https://twitter.com/VulmonFeeds/status/122013',
 'retweet': 0,
 'video': '0',
 'near': '',
 'geo': '',
 'source': '',
 'user_rt_id': 0,
 'user_rt': '',
 'retweet_id': 0,
 'reply_to': "[{'user_id': '941389496771399680', 'username'",
 'retweet_date': '',
 'transl

In [3]:
noun_list = noun
len(set(noun)), len(noun)
# noun_list = noun_listt

(16899, 60907)

In [8]:
################################################################################################
# Neo4j test for noun as node (required another db to store tweet)
# Note we only create nodes here and not the relationships
################################################################################################

from neo4j import GraphDatabase
import json
#Indicate to driver where neo4j is running
driver = GraphDatabase.driver("bolt://localhost:7687",auth=("test_user","password"), encrypted=False)

# idd = 2
with driver.session() as session:
    tx = session.begin_transaction()
    count = 0
    idd = 0
    for noun in noun_list[30000:40000]:
#         tx.run('''Match (a:Tweet {type: $word}) return a.num''', word=noun)
        result = tx.run('''Match (a:Tweet {type: $word}) return a.num''', word=noun)
        x = result.records()
        flag = True
        for i in x:
            flag = False
            tx.run('''MATCH (a:Tweet {type:$word}) SET a.num = a.num+1''', word=noun)
        if flag:
            tx.run('''CREATE (a:Tweet {type:$word, num:1, id:$idd})''', word=noun, idd=idd)
            idd += 1
        #Batch processing to run 1000 tweets as a time as these commits are quite time intensive
    count += 1
    if count > 1000:
        tx.commit()
        tx = session.begin_transaction()
        count = 0
    tx.commit()

In [54]:
from neo4j import GraphDatabase
import json
#Indicate to driver where neo4j is running
driver = GraphDatabase.driver("bolt://localhost:7687",auth=("test_user","password"), encrypted=False)

# driver = GraphDatabase.driver("bolt://localhost",auth=("test_user","password"))
x = None
with driver.session() as session:
    tx = session.begin_transaction()
    for i in range(len())
    result = tx.run('''Match (a:Tweet {type: word}) return a.num''', word=noun)
    x = result.records()

SyntaxError: invalid syntax (<ipython-input-54-1500a3907308>, line 10)

In [56]:
#####################################################################################################
# Neo4j code for forming relations (required another db to store tweet)
# Requires data to be in a 2d array. Inner array is a list of nouns/named entities in the same tweet
#####################################################################################################

from neo4j import GraphDatabase
import json
#Indicate to driver where neo4j is running
driver = GraphDatabase.driver("bolt://localhost:7687",auth=("test_user","password"), encrypted=False)

with driver.session() as session:
    tx = session.begin_transaction()
    count = 0
    for nouns in noun_from_tweet[:1000]:
        for _noun in nouns:
            tx.run('''Match (a:Tweet), (b:Tweet) where $noun = a.type and $noun = b.type
            merge (a)-[r: encountred_with {type: $noun}]->(b)''',
                   noun=_noun.upper()) # add parameters here
        #Batch processing to run 1000 tweets as a time as these commits are quite time intensive
        count += 1
        if count > 1000:
            tx.commit()
            tx = session.begin_transaction()
            count = 0
    tx.commit()

In [108]:
# print(result.single()[0])
# result.single()[0]
type(result)
# result.single()[0] == None
# result.single()
print(type(x))
for i in x:
    print('inside')
    print(i.value() == None)

<class 'generator'>


In [72]:
xx = []
for i in x:
    xx.append(i)

In [73]:
# y = hash(xx)
xx[2].value(key=1)
# xx[2]
# y[-6685369989889550852]

IndexError: list index out of range

In [ ]:
xx[2][1].items()

In [ ]:
count = 0
for i in range(len(noun)):
    for j in range(len(noun[i])):
        if ('Malware' == noun[i][j]):
            count += 1
count

In [14]:
# !python -m spacy download en
# !python -m spacy download en_core_web_sm

import spacy
nlp = spacy.load('en_core_web_sm')
sentence = "CVE-2019-897 NSA exploit EternalRomance 'formed basis for alleged Chinese tool' - iTWire Researchers Mark Lechtik and Nadav Grossman said in first offered NSA exploits that the hacking group, known variously as APT3 and Gothic Panda, had based its exploitation tool, https://ift.tt/2HRG9js"
doc = nlp(sentence)
print("The sentence is: ", sentence)
# print([ent for ent in doc.ents])
# for ent in doc.ents:
# doc
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])
for ent in doc.ents:
    print(ent.text, ent.label_)
print([chunk.text for chunk in doc.noun_chunks])
    

The sentence is:  CVE-2019-897 NSA exploit EternalRomance 'formed basis for alleged Chinese tool' - iTWire Researchers Mark Lechtik and Nadav Grossman said in first offered NSA exploits that the hacking group, known variously as APT3 and Gothic Panda, had based its exploitation tool, https://ift.tt/2HRG9js
Noun phrases: ['CVE-2019-897 NSA', "EternalRomance 'formed basis", "alleged Chinese tool' - iTWire Researchers Mark Lechtik", 'Nadav Grossman', 'the hacking group', 'APT3', 'Gothic Panda', 'its exploitation tool', 'https://ift.tt/2HRG9js']
Verbs: ['exploit', 'form', 'say', 'offer', 'exploit', 'hack', 'know', 'base']
NSA ORG
EternalRomance ORG
Chinese NORP
Mark Lechtik PERSON
Nadav Grossman PERSON
first ORDINAL
NSA ORG
Gothic Panda PERSON
['CVE-2019-897 NSA', "EternalRomance 'formed basis", "alleged Chinese tool' - iTWire Researchers Mark Lechtik", 'Nadav Grossman', 'the hacking group', 'APT3', 'Gothic Panda', 'its exploitation tool', 'https://ift.tt/2HRG9js']


In [1]:
pronouns = """all
another
any
anybody
anyone
anything
as
aught
both
each
each other
either
enough
everybody
everyone
everything
few
he
her
hers
herself
him
himself
his
I
idem
it
its
itself
many
me
mine
most
my
myself
naught
neither
no one
nobody
none
nothing
nought
one
one another
other
others
ought
our
ours
ourself
ourselves
several
she
some
somebody
someone
something
somewhat
such
suchlike
that
thee
their
theirs
theirself
theirselves
them
themself
themselves
there
these
they
thine
this
those
thou
thy
thyself
us
we
what
whatever
whatnot
whatsoever
whence
where
whereby
wherefrom
wherein
whereinto
whereof
whereon
wherever
wheresoever
whereto
whereunto
wherewith
wherewithal
whether
which
whichever
whichsoever
who
whoever
whom
whomever
whomso
whomsoever
whose
whosever
whosesoever
whoso
whosoever
ye
yon
yonder
you
your
yours
yourself
yourselves"""

extra_words = ['the']
pronouns = pronouns.split("\n")
pronouns

['all',
 'another',
 'any',
 'anybody',
 'anyone',
 'anything',
 'as',
 'aught',
 'both',
 'each',
 'each other',
 'either',
 'enough',
 'everybody',
 'everyone',
 'everything',
 'few',
 'he',
 'her',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'I',
 'idem',
 'it',
 'its',
 'itself',
 'many',
 'me',
 'mine',
 'most',
 'my',
 'myself',
 'naught',
 'neither',
 'no one',
 'nobody',
 'none',
 'nothing',
 'nought',
 'one',
 'one another',
 'other',
 'others',
 'ought',
 'our',
 'ours',
 'ourself',
 'ourselves',
 'several',
 'she',
 'some',
 'somebody',
 'someone',
 'something',
 'somewhat',
 'such',
 'suchlike',
 'that',
 'thee',
 'their',
 'theirs',
 'theirself',
 'theirselves',
 'them',
 'themself',
 'themselves',
 'there',
 'these',
 'they',
 'thine',
 'this',
 'those',
 'thou',
 'thy',
 'thyself',
 'us',
 'we',
 'what',
 'whatever',
 'whatnot',
 'whatsoever',
 'whence',
 'where',
 'whereby',
 'wherefrom',
 'wherein',
 'whereinto',
 'whereof',
 'whereon',
 'wherever',
 'wheresoever

In [18]:
sentence = "CVE-2019-897 NSA exploit EternalRomance 'formed basis for alleged Chinese tool' - iTWire Researchers Mark Lechtik and Nadav Grossman said in first offered NSA exploits that the hacking group, known variously as APT3 and Gothic Panda, had based its exploitation tool, https://ift.tt/2HRG9js"
sentence = "The texas virus incident is getting severe"
doc = nlp(sentence)
print("The sentence is: ", sentence)
for entity in doc.ents:
    print(entity.text, entity.label_)
# print([ent for ent in doc.ents])
# for ent in doc.ents:
# doc
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])


The sentence is:  The texas virus incident is getting severe
texas GPE
Noun phrases: ['The texas virus incident']


In [19]:
doc.ents

(texas,)

In [20]:
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])

Noun phrases: ['The texas virus incident']


In [21]:
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

Verbs: ['get']


In [22]:
for entity in doc.ents:
    print(entity.text, entity.label_)

texas GPE


In [ ]:
doc.ents

In [ ]:
# oauth_consumer_key - ZYOj2R1VYsIUhaRvsrcRuHxk6
# oauth_nonce	kYjzVBB8Y0ZFabxSWbWovY3uYSQ2pTgmZeNu2VS4cg
# Consumer Secret = API Secret Key = uTwMxW2CPXfv8wZCcqOSX8Yw46eqJVMNopqh5kh9O0VXiVQNRG
# Access token secret = OAuth token secret: VB6fxb1akdtwRDf52sbQlPKE9tpHEPX1ZBSI3Y1t33REk
# Signing key = uTwMxW2CPXfv8wZCcqOSX8Yw46eqJVMNopqh5kh9O0VXiVQNRG&VB6fxb1akdtwRDf52sbQlPKE9tpHEPX1ZBSI3Y1t33REk
# oauth-token = 797644398670409728-I3aUqpmhD7uPscfFZFxMGfMWXPBmiaN
# oauth_signature = Use HMAC
# curl -XPOST 
#   --url 'https://api.twitter.com/1.1/statuses/update.json?status=hello' 
#   --header 'authorization: OAuth
#   oauth_consumer_key="oauth_customer_key",
#   oauth_nonce="generated_oauth_nonce",
#   oauth_signature="generated_oauth_signature",
#   oauth_signature_method="HMAC-SHA1",
#   oauth_timestamp="generated_timestamp",
#   oauth_token="797644398670409728-I3aUqpmhD7uPscfFZFxMGfMWXPBmiaN",
#   oauth_version="1.0"'

# NEW KEYS: (DO NOT SHARE)
# Access token :797644398670409728-Zwgcl9kcCFerhFNlFFGwR3emSbfpfpX
# Copy
# Access token secret :CzVCCqD8X9FC059X98deDiNYb24IWjhZYVeAhoU4F5v7l
# Copy


In [1]:
#####################################################################################################
# Twitter API Connection established
#####################################################################################################

import twitter
api = twitter.Api(consumer_key="ZYOj2R1VYsIUhaRvsrcRuHxk6",
                  consumer_secret="uTwMxW2CPXfv8wZCcqOSX8Yw46eqJVMNopqh5kh9O0VXiVQNRG",
                  access_token_key="797644398670409728-Zwgcl9kcCFerhFNlFFGwR3emSbfpfpX",
                  access_token_secret="CzVCCqD8X9FC059X98deDiNYb24IWjhZYVeAhoU4F5v7l")

In [59]:
results = api.GetSearch(
    raw_query="q=from%3Atwitterdev&result_type=mixed&count=2&tweet_mode=extended")

In [60]:
# results[27].AsJsonString
x = results[0].AsDict()

In [64]:
#  dir(twitter.models.Status)
# x

In [62]:
>>> api.GetSearch(geocode=[37.781157, -122.398720, "1mi"])

[Status(ID=1238217454226042880, ScreenName=BloomTV, Created=Thu Mar 12 21:37:00 +0000 2020, Text='Downtown San Francisco is so quiet it feels like a Sunday. At every table I pass, I hear conversation about the… https://t.co/BBVTjPL1pd'),
 Status(ID=1238280494011334658, ScreenName=SFPublicLibrary, Created=Fri Mar 13 01:47:30 +0000 2020, Text='We know it’s a scary time, but don’t lose your #Census. \n\nThe 2020 Census kicked off today, and we’re joining the e… https://t.co/jEmOKbCSgp'),
 Status(ID=1238398634766581761, ScreenName=20thcenturygoth, Created=Fri Mar 13 09:36:57 +0000 2020, Text='The Office\n\nWow!\n\n#theoffice #dwight #pepperspray #pam #jim #roy @ San Francisco, California, U.S.A https://t.co/6bvNCb3ddI')]

In [63]:
x

{'created_at': 'Tue Mar 10 17:47:52 +0000 2020',
 'favorite_count': 78,
 'full_text': 'Today we’re introducing a simplified &amp; more straightforward set of rules for developers. This policy is an important part of our commitment to protecting the safety &amp; privacy of the people who use our service. https://t.co/2ZYxJ2jOOO\n\nRead on for highlights from the policy \U0001f9f5',
 'hashtags': [],
 'id': 1237435016134656006,
 'id_str': '1237435016134656006',
 'lang': 'en',
 'retweet_count': 39,
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'urls': [{'expanded_url': 'https://blog.twitter.com/developer/en_us/topics/community/2020/twitter_developer_policy_update.html',
   'url': 'https://t.co/2ZYxJ2jOOO'}],
 'user': {'created_at': 'Sat Dec 14 04:35:55 +0000 2013',
  'description': "The voice of Twitter's #DevRel team, and your official source for updates, news, & events about Twitter's API.\n\nNeed help? Visit https://t.co/DVDf7qKyS9",
  'favourite